In [60]:
import copy

class Product:
    def __init__(self, name, price, available, quantity):
        self.name = name
        self.price = price
        self.available = available
        self.quantity = quantity

class CartItem:
    def __init__(self, product, quantity):
        self.product = product
        self.quantity = quantity

class CartPrototype:
    def clone(self):
        return copy.deepcopy(self)

class DiscountStrategy:
    def apply_discount(self, price):
        pass

class PercentageDiscount(DiscountStrategy):
    def __init__(self, percentage):
        self.percentage = percentage

    def apply_discount(self, price):
        return price - (price * self.percentage / 100)

class BuyOneGetOneFree(DiscountStrategy):
    def apply_discount(self, price):
        return price * 0.5  # 50% off for buy one get one free

class Cart:
    def __init__(self, products, discount_strategy=None):
        self.items = []
        self.discount_strategy = discount_strategy
        self.products = products  # Pass the products list to the Cart class

    def add_item(self, item):
        for existing_item in self.items:
            if existing_item.product.name == item.product.name:
                existing_item.quantity += item.quantity
                return
        self.items.append(item)

    def update_quantity(self, product_name, new_quantity):
        for item in self.items:
            if item.product.name == product_name:
                item.quantity = new_quantity
                return

    def remove_item(self, product_name):
        self.items = [item for item in self.items if item.product.name != product_name]

    def view_cart(self):
        if not self.items:
            print("Your cart is empty.")
        else:
            print("Items in your cart:")
            for item in self.items:
                print(f"{item.quantity} {item.product.name} - ${item.product.price} each")

    def calculate_total(self):
        total = sum(item.product.price * item.quantity for item in self.items)
        return total

    def apply_discount(self, total, discount_strategy=None):
        if discount_strategy:
            return discount_strategy.apply_discount(total)
        elif self.discount_strategy:
            return self.discount_strategy.apply_discount(total)
        else:
            return total

    def reset_cart(self):
        self.items = []

    def update_available_quantity(self, product_name, new_quantity):
        for product in self.products:
            if product.name == product_name:
                product.quantity -= new_quantity
                return

    def display_available_products(self):
        print("Available Products:")
        for product in self.products:
            print(f"{product.name} - ${product.price} - Quantity: {product.quantity} - {'Available' if product.available else 'Out of stock'}")

    def display_available_discounts(self, available_discounts):
        print("Available Discounts:")
        for idx, discount in enumerate(available_discounts, start=1):
            print(f"{idx}. {discount['name']} - {discount['description']}")

    def choose_discount(self, available_discounts, product_name):
        # Choose discount based on product type
        if product_name == "Laptop":
            return next((discount['strategy'] for discount in available_discounts if discount['name'] == "Percentage Discount"), None)
        elif product_name == "Headphones":
            return next((discount['strategy'] for discount in available_discounts if discount['name'] == "Buy One Get One Free"), None)
        else:
            return None

def main():
    # Example Products
    products = [Product("Laptop", 1000, True, 5), Product("Headphones", 50, True, 10)]

    # Create shopping cart without a specific discount strategy
    cart = Cart(products)

    while True:
        action = input("\nWhat would you like to do? (add/update/remove/view/checkout/exit): ").lower()

        if action == 'exit':
            break
        elif action == 'add':
            cart.display_available_products()
            product_name = input("Enter the name of the product you want to add to the cart: ")
            quantity = int(input("Enter the quantity: "))

            # Check if the product is Headphones and quantity is multiples of 2
            if product_name == "Headphones" and quantity % 2 != 0:
                print("Error: order quantity for Headphones is multiples of 2 due to one plus one offer.")
                continue

            for product in products:
                if (
                    product.name == product_name
                    and product.available
                    and quantity <= product.quantity
                ):
                    # Use the prototype pattern to clone the product
                    product_clone = copy.deepcopy(product)
                    product_clone.quantity = quantity  # Set the quantity for the cloned product

                    # Choose discount based on product type
                    discount_strategy = cart.choose_discount([
                        {"name": "Percentage Discount", "strategy": PercentageDiscount(10), "description": "10% off"},
                        {"name": "Buy One Get One Free", "strategy": BuyOneGetOneFree(), "description": "Buy one, get one free"}
                    ], product_name)

                    cart.add_item(CartItem(product_clone, quantity))
                    print(f"{quantity} {product.name}(s) added to the cart.")

                    # Update the available quantity in the product list
                    cart.update_available_quantity(product_name, quantity)

                    break
            else:
                if not any(product.name == product_name and product.available for product in products):
                    print("Error: Product not found or not available.")
                elif quantity > next((product.quantity for product in products if product.name == product_name), 0):
                    print("Error: Insufficient quantity.")
        elif action == 'update':
            cart.view_cart()
            product_name = input("Enter the name of the product you want to update: ")
            new_quantity = int(input("Enter the new quantity: "))
            cart.update_quantity(product_name, new_quantity)
            print(f"Quantity of {product_name} updated to {new_quantity}.")
        elif action == 'remove':
            product_name = input("Enter the name of the product you want to remove from the cart: ")
            cart.remove_item(product_name)
            print(f"{product_name} removed from the cart.")
        elif action == 'view':
            cart.view_cart()
        elif action == 'checkout':
            cart.view_cart()
            total_bill = cart.calculate_total()

            # Display available discount options
            available_discounts = [
                {"name": "Percentage Discount", "strategy": PercentageDiscount(10), "description": "10% off"},
                {"name": "Buy One Get One Free", "strategy": BuyOneGetOneFree(), "description": "Buy one, get one free"}
                # Add more discount options as needed
            ]

            # Display available discounts
            cart.display_available_discounts(available_discounts)

            # Apply the chosen discount and decrement available quantity
            for item in cart.items:
                chosen_discount = cart.choose_discount(available_discounts, item.product.name)
                if chosen_discount:
                    discount_amount = item.product.price - chosen_discount.apply_discount(item.product.price)
                    print(f"Applied {chosen_discount.__class__.__name__} to {item.product.name}: -${discount_amount:.2f}")

                    item.product.price = chosen_discount.apply_discount(item.product.price)


            discounted_total = cart.calculate_total()
            print(f"\nDiscounted total: ${discounted_total:.2f}")

            # Print the final total
            print(f"Final Total: ${discounted_total:.2f}")

            # Reset the cart after checkout
            cart.reset_cart()

if __name__ == "__main__":
    main()


What would you like to do? (add/update/remove/view/checkout/exit): add
Available Products:
Laptop - $1000 - Quantity: 5 - Available
Headphones - $50 - Quantity: 10 - Available
Enter the name of the product you want to add to the cart: Laptop
Enter the quantity: 2
2 Laptop(s) added to the cart.

What would you like to do? (add/update/remove/view/checkout/exit): add
Available Products:
Laptop - $1000 - Quantity: 3 - Available
Headphones - $50 - Quantity: 10 - Available
Enter the name of the product you want to add to the cart: Headphones
Enter the quantity: 4
4 Headphones(s) added to the cart.

What would you like to do? (add/update/remove/view/checkout/exit): checkout
Items in your cart:
2 Laptop - $1000 each
4 Headphones - $50 each
Available Discounts:
1. Percentage Discount - 10% off
2. Buy One Get One Free - Buy one, get one free
Applied PercentageDiscount to Laptop: -$100.00
Applied BuyOneGetOneFree to Headphones: -$25.00

Discounted total: $1900.00
Final Total: $1900.00

What woul